In [11]:
#Import the libraries
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [12]:
#Create a 2-D feature NumPy array with random integers

features = (np.random.randint(10, size=(100, 1)))
print(features.shape)

(100, 1)


In [13]:
# Split the dataset into 75/25 for train and test.

training_dataset_length = math.ceil(len(features) * .75)
print(training_dataset_length)

75


Preprocess the data i.e feature scaling to scale the data to be valued in between 0 and 1, which is a good practice to scale the data before feeding it into a neural network for optimal performance.

In [14]:
#Scale the all of the data to be values between 0 and 1 

scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(features)

Here we predict the 11th values using [1,2,….,10] values and so on. 

Here **N = 100** and size of the sliding window is l = 10. 

So **x_train** will contain values of sliding windows of l = 10 and y_train will contain values of every l+1 value which we want to predict.

In [15]:
train_data = scaled_data[0:training_dataset_length  , : ]

#Splitting the data
x_train=[]
y_train = []

for i in range(10, len(train_data)):
    x_train.append(train_data[i-10:i,0])
    y_train.append(train_data[i,0])

Then converting the x_train and y_train into NumPy array values and reshaping it into a 3-D array, shape accepted by the LSTM model.

In [16]:
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

Build the architecture


* Make an object of the sequential model. Then add the LSTM layer with parameters (units: the dimension of output space, input_shape: the shape of the training set, return_sequences: Ture or False, determines whether to return the last output in the output sequence or the full sequence.

* We add 4 of the LSTM layers each with a dropout layer of value(0.2).

{Droupout layer is a type of regularization technique which is used to prevent overfitting, but it may also increase training time in some cases.}

* Final layer is the output layer which is a fully connected dense layer(units = 1, as we are predicting only one value i.e l+1).

{Dense layer performs the operation on the input layers and returns the output and every neuron at the previous layer is connected to the neurons in the next layer hence it is called fully connected Dense layer.}

In [17]:
from keras.layers import Dropout

# Initialising the RNN
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units = 1))

Compile the model using ‘adam optimizer’ (It is a learning rate optimization algorithm used while training of DNN models) and error is calculated by loss function ‘mean squared error’ ( as it is a regression problem so we use mean squared error loss function).


Then fit the model on 30 epoch(epochs are the number of times we pass the data into the neural network) and a batch size of 50(we pass the data in batches, segmenting the data into smaller parts so as for network to process the data in parts).

In [18]:
#compile and fit the model on 30 epochs

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 30, batch_size = 50)

Epoch 1/30
2/2 [==============================] - 6s 23ms/step - loss: 0.4324
Epoch 2/30
2/2 [==============================] - 0s 24ms/step - loss: 0.3716
Epoch 3/30
2/2 [==============================] - 0s 26ms/step - loss: 0.3014
Epoch 4/30
2/2 [==============================] - 0s 23ms/step - loss: 0.2210
Epoch 5/30
2/2 [==============================] - 0s 22ms/step - loss: 0.1341
Epoch 6/30
2/2 [==============================] - 0s 22ms/step - loss: 0.1263
Epoch 7/30
2/2 [==============================] - 0s 22ms/step - loss: 0.1812
Epoch 8/30
2/2 [==============================] - 0s 23ms/step - loss: 0.1508
Epoch 9/30
2/2 [==============================] - 0s 24ms/step - loss: 0.1145
Epoch 10/30
2/2 [==============================] - 0s 24ms/step - loss: 0.1092
Epoch 11/30
2/2 [==============================] - 0s 22ms/step - loss: 0.1264
Epoch 12/30
2/2 [==============================] - 0s 22ms/step - loss: 0.1227
Epoch 13/30
2/2 [==============================] - 0s 27ms/st

Create test data similar to train data, convert to NumPy array and reshape the array to 3-D shape.

In [19]:
#Test data set
test_data = scaled_data[training_dataset_length - 10: , : ]

#splitting the x_test and y_test data sets
x_test = []
y_test =  features[training_dataset_length : , : ] 

for i in range(10,len(test_data)):
    x_test.append(test_data[i-10:i,0])
    
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

Making the predictions and calculating the rmse score(smaller the rmse score, better the model has performed).

In [20]:
#check predicted values
predictions = model.predict(x_test) 
#Undo scaling
predictions = scaler.inverse_transform(predictions)

#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

2.6284689347641246